In [1]:
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
sqlite_file = "FPA_FOD_20170508.sqlite"
conn=sqlite3.connect(sqlite_file)
c = conn.cursor()
fires = pd.read_sql_query("SELECT * FROM fires", conn)

In [3]:
#Julian Date format to Gregorian dates
fires['DATE'] = pd.to_datetime(fires['DISCOVERY_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
fires['MONTH'] = pd.DatetimeIndex(fires['DATE']).month
fires['DAY_OF_WEEK'] = fires['DATE'].dt.weekday_name

In [4]:
#Add wildfires duration 
fires['DURATION_DAYS'] = fires['CONT_DOY'] - fires['DISCOVERY_DOY']

In [5]:
#fires.to_csv('finalized_data_Uncategorized_april_27_2019.csv')

In [6]:
fires.columns

Index(['OBJECTID', 'FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM',
       'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID',
       'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT',
       'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID',
       'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME',
       'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME',
       'COMPLEX_NAME', 'FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY',
       'DISCOVERY_TIME', 'STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR', 'CONT_DATE',
       'CONT_DOY', 'CONT_TIME', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE',
       'LONGITUDE', 'OWNER_CODE', 'OWNER_DESCR', 'STATE', 'COUNTY',
       'FIPS_CODE', 'FIPS_NAME', 'Shape', 'DATE', 'MONTH', 'DAY_OF_WEEK',
       'DURATION_DAYS'],
      dtype='object')

In [7]:
fires

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape,DATE,MONTH,DAY_OF_WEEK,DURATION_DAYS
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...,2005-02-02,2,Wednesday,0.0
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...,2004-05-12,5,Wednesday,0.0
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,STATE OR PRIVATE,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...,2004-05-31,5,Monday,0.0
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...,2004-06-28,6,Monday,5.0
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...,2004-06-28,6,Monday,5.0
5,6,6,FS-1418849,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,USFS,CA,5,005,Amador,b'\x00\x01\xad\x10\x00\x00\xf0<~\x90\xa1\x06^\...,2004-06-30,6,Wednesday,1.0
6,7,7,FS-1418851,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,USFS,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00$o\x996\xd0\t^\xc0h\...,2004-07-01,7,Thursday,1.0
7,8,8,FS-1418854,FED,FS-FIRESTAT,FS,USCASHF,Shasta-Trinity National Forest,0514,Shasta-Trinity National Forest,...,STATE OR PRIVATE,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00t)\xe8\xd5\xc4\x9b^\...,2005-03-08,3,Tuesday,0.0
8,9,9,FS-1418856,FED,FS-FIRESTAT,FS,USCASHF,Shasta-Trinity National Forest,0514,Shasta-Trinity National Forest,...,STATE OR PRIVATE,CA,None,None,None,"b'\x00\x01\xad\x10\x00\x00\xdc\x8d\x1e""""\x92^\...",2005-03-15,3,Tuesday,0.0
9,10,10,FS-1418859,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,USFS,CA,5,005,Amador,b'\x00\x01\xad\x10\x00\x00dS\\\xf2\x8b\t^\xc0\...,2004-07-01,7,Thursday,1.0


In [9]:
df = fires[['STAT_CAUSE_DESCR','FIRE_YEAR','COUNTY','LATITUDE','LONGITUDE','FIRE_SIZE',
            'FIRE_SIZE_CLASS','MONTH','DAY_OF_WEEK','STATE','DATE','OWNER_CODE',
            'OWNER_DESCR','DURATION_DAYS']]

In [10]:
def cat3 (cats):
    if cats in ['Lightning','Structure','Powerline','Railroad']:
        return 'accident'
    elif cats in ['Arson','Smoking','Children','Campfire','Equipment Use','Debris Burning','Fireworks']:
        return 'human_caused'
    else:
        return 'other'

# Make two dataframes 
df3cats = df.copy() # make a copy to keep things neat
df3cats['STAT_CAUSE_DESCR'] = df3cats['STAT_CAUSE_DESCR'].apply(cat3)


categorizeCAUSE3Cats = df3cats.groupby('STAT_CAUSE_DESCR').STAT_CAUSE_DESCR.count()
print('three Categories: ')
print(categorizeCAUSE3Cats.head())

#df3cats.head(10)

three Categories: 
STAT_CAUSE_DESCR
accident         330167
human_caused    1059770
other            490528
Name: STAT_CAUSE_DESCR, dtype: int64


In [11]:
df3cats.head(10)

,STAT_CAUSE_DESCR,FIRE_YEAR,COUNTY,LATITUDE,LONGITUDE,FIRE_SIZE,FIRE_SIZE_CLASS,MONTH,DAY_OF_WEEK,STATE,DATE,OWNER_CODE,OWNER_DESCR,DURATION_DAYS
0,other,2005,63,40.036944,-121.005833,0.10,A,2,Wednesday,CA,2005-02-02,5.0,USFS,0.0
1,accident,2004,61,38.933056,-120.404444,0.25,A,5,Wednesday,CA,2004-05-12,5.0,USFS,0.0
2,human_caused,2004,17,38.984167,-120.735556,0.10,A,5,Monday,CA,2004-05-31,13.0,STATE OR PRIVATE,0.0
3,accident,2004,3,38.559167,-119.913333,0.10,A,6,Monday,CA,2004-06-28,5.0,USFS,5.0
4,accident,2004,3,38.559167,-119.933056,0.10,A,6,Monday,CA,2004-06-28,5.0,USFS,5.0
5,accident,2004,5,38.635278,-120.103611,0.10,A,6,Wednesday,CA,2004-06-30,5.0,USFS,1.0
6,accident,2004,17,38.688333,-120.153333,0.10,A,7,Thursday,CA,2004-07-01,5.0,USFS,1.0
7,human_caused,2005,None,40.968056,-122.433889,0.80,B,3,Tuesday,CA,2005-03-08,13.0,STATE OR PRIVATE,0.0
8,human_caused,2005,None,41.233611,-122.283333,1.00,B,3,Tuesday,CA,2005-03-15,13.0,STATE OR PRIVATE,0.0
9,accident,2004,5,38.548333,-120.149167,0.10,A,7,Thursday,CA,2004-07-01,5.0,USFS,1.0


In [12]:
#df3cats.to_csv('finalized_data_categorized_april_27_2019.csv')